In [5]:
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
from collections import defaultdict

In [6]:
age_begin = 13
age_end = 100
risk_groups = ['HM', 'HF', 'MSM']
num_risk = 3

all_jur = 10
age_groups = 88
number_of_risk_groups = 3
number_of_compartments = 22
dt = 1/12

num_age = 88
num_comp = number_of_compartments-2

prep_efficiency = 0.99

unaware_index = (1,5,9,13,17)
aware_no_care_index = (2,6,10,14,18)
ART_VLS_index = (3,4,7,8,11,12,15,16,19,20)
VLS_index = (4,8,12,16,20)


pop_growth_rate = 0

gamma = np.array([[0.5,0.5,0.5,0.5,1],
                  [0.5,0.5,0.5,0.5,1],
                  [0.5,0.5,0.5,0.5,1]])

scaling_factor_dropout = np.array([[1,1,1,1,1,1,1,1,0,0],
                                   [1,1,1,1,1,1,1,1,0,0],
                                   [1,1,1,1,1,1,1,1,0,0]])

all_jurisdictions = [6001,6037,6059,6065,6067,6071,6073,6]

num_jur = len(all_jurisdictions)

cluster1 = [6001]
cluster2 = [6037]
cluster3 = [6059]
cluster4 = [6065]
cluster5 = [6067]
cluster6 = [6071]
cluster7 = [6073]
cluster8 = [6]

cluster1_index = np.array([0])
cluster2_index = np.array([1])
cluster3_index = np.array([2])
cluster4_index = np.array([3])
cluster5_index = np.array([4])
cluster6_index = np.array([5])
cluster7_index = np.array([6])
cluster8_index = np.array([7])

In [7]:
from ipynb.fs.full.reading_files import read_new_inf_input

In [8]:
new_infections_data = read_new_inf_input(age_begin, age_end, risk_groups)

In [ ]:
mixing_excel = './input_files/JURI_mixing_weightedBydistance-6-3-2021.xlsx'

mixing_df_hm = pd.read_excel(mixing_excel, sheet_name='HETM_mixing')
mixing_df_hf = pd.read_excel(mixing_excel, sheet_name='HETF_mixing')
mixing_df_msm = pd.read_excel(mixing_excel, sheet_name='MSM_mixing')

mixing_df_hm = mixing_df_hm[['FIPS', 6001,6037,6059,6065,6067,6071,6073,6]]
mixing_df_hf = mixing_df_hf[['FIPS', 6001,6037,6059,6065,6067,6071,6073,6]]
mixing_df_msm = mixing_df_msm[['FIPS', 6001,6037,6059,6065,6067,6071,6073,6]]

mixing_hm = mixing_df_hm.loc[mixing_df_hm['FIPS'].isin(all_jurisdictions)]
mixing_hf = mixing_df_hf.loc[mixing_df_hf['FIPS'].isin(all_jurisdictions)]
mixing_msm = mixing_df_msm.loc[mixing_df_msm['FIPS'].isin(all_jurisdictions)]

hm_array = mixing_hm.values[:,1:]
hf_array = mixing_hf.values[:,1:]
msm_array = mixing_msm.values[:,1:]

hm_sum = np.sum(hm_array, axis=1)
hf_sum = np.sum(hf_array, axis=1)
msm_sum = np.sum(msm_array, axis=1)

hm_array_scaled = hm_array / hm_sum[:,np.newaxis]
hf_array_scaled = hf_array / hf_sum[:,np.newaxis]
msm_array_scaled = msm_array / msm_sum[:,np.newaxis]

mixing_matrix = np.zeros((num_risk,len(all_jurisdictions), len(all_jurisdictions)))

mixing_matrix[0,:,:] = hm_array_scaled[:,:]
mixing_matrix[1,:,:] = hf_array_scaled[:,:]
mixing_matrix[2,:,:] = msm_array_scaled[:,:]

In [ ]:
def new_infections_per_month(num_jur, data_array, new_infections_data, M_x1_y1_i, prep_risk):
    
    risk_mat = new_infections_data["sex_mixing"].copy()[0:num_risk,0:num_risk]
    age_mat = new_infections_data["age_mixing_final_mat"].copy()
    
    I = data_array[:,:,:,1:21]
    N = np.sum(data_array[:,:,:,0:21], axis=3)
    d_x1_y1 = new_infections_data["num_partner_risk_casual"].copy()+new_infections_data["num_partner_risk_casual_only"].copy()
    sus_x1_y1 = data_array[:,:,:,0]
    mat_vector = np.repeat(age_mat[:,np.newaxis,:,:],num_risk, axis = 1) * risk_mat[:,:,np.newaxis, np.newaxis]
    I_N_vector = I / N[:,:,:,np.newaxis]
    I_N_mult_vector = mat_vector[np.newaxis,:,:,:,:,np.newaxis]*I_N_vector[:,np.newaxis,:,np.newaxis,:,:]
    Q_inner_vector = np.apply_over_axes(np.sum, I_N_mult_vector, [2,4]).reshape((num_jur, num_risk,num_age,num_comp))
    q_x_y_i_vector = d_x1_y1[np.newaxis,:,np.newaxis,np.newaxis]*dt*Q_inner_vector
    q_mix_vector = np.zeros((num_jur,num_jur,num_risk,num_age,num_comp))
    for risk in range(num_risk):
        q_mix_vector[:,:,risk,:,:] = mixing_matrix[risk][:,:,np.newaxis,np.newaxis]*q_x_y_i_vector[:,risk,:,:][np.newaxis,:,:,:]
    q_mix_sum_vector = np.sum(q_mix_vector, axis = 1)
    M_power_vector = M_x1_y1_i[np.newaxis,:,:,:]**q_mix_sum_vector
    M_prod_vector = 1-np.prod(M_power_vector, axis = 3) 
    
    new_inf_per_month = sus_x1_y1*(1 - prep_risk[:,:,np.newaxis])*M_prod_vector
    
    return new_inf_per_month

In [ ]:
def calculate_proportions(data_array, num_jur, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index,VLS_index):
    
    plwh_risk = np.zeros((num_jur, number_of_risk_groups))
    unaware_risk = np.zeros((num_jur, number_of_risk_groups))
    aware_no_art_risk = np.zeros((num_jur, number_of_risk_groups))
    aware_art_vls_risk = np.zeros((num_jur, number_of_risk_groups))
    vls_risk = np.zeros((num_jur, number_of_risk_groups))
    
    for risk in range(number_of_risk_groups):
        plwh_risk[:,risk] = np.apply_over_axes(np.sum, data_array[:,risk,:,1:21], [1,2]).reshape(num_jur,)
#         print('risk')
#         print(plwh_risk[risk])
        unaware_risk[:,risk] = np.apply_over_axes(np.sum, data_array[:,risk,:,unaware_index], [0,2]).reshape(num_jur,)
#         print('unaware')
#         print(unaware_risk[risk])
        aware_no_art_risk[:,risk] = np.apply_over_axes(np.sum, data_array[:,risk,:,aware_no_care_index], [0,2]).reshape(num_jur,)
#         print('aware_no_art')
#         print(aware_no_art_risk[risk])
        aware_art_vls_risk[:,risk] = np.apply_over_axes(np.sum, data_array[:,risk,:,ART_VLS_index], [0,2]).reshape(num_jur,)
#         print('art vls')
#         print(aware_art_vls_risk[risk])

        vls_risk[:,risk] = np.apply_over_axes(np.sum, data_array[:,risk,:,VLS_index], [0,2]).reshape(num_jur,)
        
    
    total_pop = np.apply_over_axes(np.sum, data_array[:,:,:,0:21], [1,2,3]).reshape(num_jur,1)
    
    prevalence_prop = plwh_risk/total_pop
    
    unaware_prop = unaware_risk/plwh_risk
    aware_no_art_prop = aware_no_art_risk/plwh_risk
    aware_art_vls_prop = aware_art_vls_risk/plwh_risk
    vls_prop = vls_risk/plwh_risk
    
    return total_pop, np.round(prevalence_prop, 6), np.round(unaware_prop, 6), \
                np.round(aware_no_art_prop, 6), np.round(aware_art_vls_prop, 6), vls_prop

In [ ]:
def diagnosis_rate(data_array, num_jur, a_unaware, unaware_index, number_of_risk_groups, new_inf_per_month, unaware_prop, death_per_month_risk_age_compartments):
    
    diagnosis_rate_risk = np.zeros((num_jur, number_of_risk_groups))
    
    a_unaware_t = np.round(a_unaware*dt, 6)

    for risk in range(len(risk_groups)):
        # new infectiion per month
        A = np.sum(new_inf_per_month, axis=2)[:,risk]

        # number of unaware population
        B = np.apply_over_axes(np.sum, data_array[:,risk,:,unaware_index], [0,2]).reshape(num_jur,)

        #number of unaware next time period
        # (current inf + new inf - total death)
        C = (np.apply_over_axes(np.sum, data_array[:,risk,:,1:21], [1,2]).reshape(num_jur,) + A - np.apply_over_axes(np.sum, death_per_month_risk_age_compartments[:,risk,:,1:21],[1,2]).reshape(num_jur,)) * (unaware_prop[:,risk] + a_unaware_t[:,risk])

        # total deaths in each compartment
        D = np.apply_over_axes(np.sum, death_per_month_risk_age_compartments[:,risk,:,unaware_index],[0,2]).reshape(num_jur,)

        # number of people in unaware compartment
        E = np.sum(np.sum(data_array[:,risk,:, unaware_index],axis=2)*new_infections_data["testing_mult_fac_risk"][risk].reshape(5,1), axis=0)
        
        diagnosis_rate_risk[:,risk] = (A+B-C-D)/E
        
        diagnosis_rate_risk[:,risk][diagnosis_rate_risk[:,risk] < 0] = 0
        
    return diagnosis_rate_risk

In [ ]:
def dropout_rate(num_jur, a_art, ART_VLS_index, diagnosis_rate_risk, ltc_risk, gamma, number_of_risk_groups, data_array, new_inf_per_month, unaware_prop, aware_no_art_prop, aware_art_vls_prop, death_per_month_risk_age_compartments):
    
    dropout_rate_risk = np.zeros((num_jur, number_of_risk_groups))
    
    a_art_t = np.round(a_art *dt, 6)
    gamma_t = np.round(gamma *dt, 6)
    
    for risk in range(len(risk_groups)):
       # total art vls pop 
        F = np.apply_over_axes(np.sum, data_array[:,risk,:,ART_VLS_index], [0,2]).reshape(num_jur,)
        #multiply F with phi for denominator
        
        K = np.sum(np.sum(data_array[:,risk,:,ART_VLS_index], axis=2)*scaling_factor_dropout[risk].reshape(10,1), axis=0)        
        # diagnosed and linked to care
        
        G = diagnosis_rate_risk[:,risk]*ltc_risk[:,risk]*np.sum((np.sum(data_array[:,risk,:,unaware_index],axis=2))*new_infections_data["testing_mult_fac_risk"][risk].reshape(5,1), axis=0)

        #entering care from unaware
        H = np.sum(gamma_t[risk].reshape(5,1)*np.sum(data_array[:,risk,:,aware_no_care_index], axis=2), axis=0)

        #total death art vls
        I = np.apply_over_axes(np.sum,death_per_month_risk_age_compartments[:,risk,:,ART_VLS_index],[0,2]).reshape(num_jur,)

        #number of art vls next time period
        J = (np.apply_over_axes(np.sum, data_array[:,risk,:,1:21], [1,2]).reshape(num_jur,) + np.sum(new_inf_per_month, axis=2)[:,risk] - np.apply_over_axes(np.sum, death_per_month_risk_age_compartments[:,risk,:,1:21],[1,2]).reshape(num_jur,)) * (aware_art_vls_prop[:,risk] + a_art_t[:,risk])
        
        dropout_rate_risk[:,risk] = (F+G+H-I-J)/K
        
#         if dropout_rate_risk[:,risk].any() < 0:
        dropout_rate_risk[:,risk][dropout_rate_risk[:,risk] < 0] = 0
        
    return dropout_rate_risk

In [ ]:
def q_matrix(num_jur, new_infections_data, diagnosis_rate_risk, dropout_rate_risk, ltc_risk):
    
    Q_MAT = new_infections_data['q']
    Q_matrix = np.zeros((num_jur, number_of_risk_groups, number_of_compartments, number_of_compartments))
    
    for jur in range(num_jur):
        for risk in range(number_of_risk_groups):
            Q_mat = Q_MAT.copy()
            Q_mat[np.where(Q_mat == 12345)] = (1 - ltc_risk[jur,risk]) * diagnosis_rate_risk[jur,risk]*new_infections_data["testing_mult_fac_risk"][risk]
            Q_mat[np.where(Q_mat == 123456)] = dropout_rate_risk[jur,risk]
            Q_mat[np.where(Q_mat == 1234567)] = ltc_risk[jur,risk] * diagnosis_rate_risk[jur,risk]*new_infections_data["testing_mult_fac_risk"][risk]

            Q_matrix[jur,risk] = Q_mat
            
        
    return Q_matrix  

In [ ]:
def q_mat_diag(Q_matrix, num_jur):
    
    Q_matrix_diagonal = np.zeros((num_jur, number_of_risk_groups, number_of_compartments, number_of_compartments))
    
    for jur in range(num_jur):
        for risk in range(number_of_risk_groups):
            Q_i = Q_matrix[jur,risk].copy()
            Q_i_sum = np.sum(Q_i, 1)
            Q_matrix_diagonal[jur,risk] = np.diag(Q_i_sum)
        
    return Q_matrix_diagonal

In [ ]:
# pop_susceptible_12_years = data_array_cluster[:,:,0,:]

def aging(data_array, pop_susceptible_12_years):
    new_pop = np.zeros((num_jur, number_of_risk_groups, num_age, number_of_compartments))
    
    new_pop[:,:,1:,:] = data_array[:,:,0:num_age-1,:]
    new_pop[:,:,0,0] = pop_susceptible_12_years
    
    return new_pop

In [ ]:
def extract_state(data_array, prep_values):
    
    data_array_cluster1 = data_array[cluster1_index,:,:,:]
    data_array_cluster2 = data_array[cluster2_index,:,:,:]
    data_array_cluster3 = data_array[cluster3_index,:,:,:]
    data_array_cluster4 = data_array[cluster4_index,:,:,:]
    data_array_cluster5 = data_array[cluster5_index,:,:,:]
    data_array_cluster6 = data_array[cluster6_index,:,:,:]
    data_array_cluster7 = data_array[cluster7_index,:,:,:]
    data_array_cluster8 = data_array[cluster8_index,:,:,:]
    
    #prep_rate
    total_data_cluster1 = np.sum(data_array_cluster1, axis=0)
    total_data_cluster1 = total_data_cluster1[np.newaxis,:,:,:]
    
    total_data_cluster2 = np.sum(data_array_cluster2, axis=0)
    total_data_cluster2 = total_data_cluster2[np.newaxis,:,:,:]
    
    total_data_cluster3 = np.sum(data_array_cluster3, axis=0)
    total_data_cluster3 = total_data_cluster3[np.newaxis,:,:,:]
    
    total_data_cluster4 = np.sum(data_array_cluster4, axis=0)
    total_data_cluster4 = total_data_cluster4[np.newaxis,:,:,:]
    
    total_data_cluster5 = np.sum(data_array_cluster5, axis=0)
    total_data_cluster5 = total_data_cluster5[np.newaxis,:,:,:]
    
    total_data_cluster6 = np.sum(data_array_cluster6, axis=0)
    total_data_cluster6 = total_data_cluster6[np.newaxis,:,:,:]
    
    total_data_cluster7 = np.sum(data_array_cluster7, axis=0)
    total_data_cluster7 = total_data_cluster7[np.newaxis,:,:,:]
    
    total_data_cluster8 = np.sum(data_array_cluster8, axis=0)
    total_data_cluster8 = total_data_cluster8[np.newaxis,:,:,:]

    
    total_pop1, prevalence_prop1, unaware_prop1, aware_no_art_prop1, aware_art_vls_prop1,_ = calculate_proportions(total_data_cluster1, 1, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)
    total_pop2, prevalence_prop2, unaware_prop2, aware_no_art_prop2, aware_art_vls_prop2,_ = calculate_proportions(total_data_cluster2, 1, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)
    total_pop3, prevalence_prop3, unaware_prop3, aware_no_art_prop3, aware_art_vls_prop3,_ = calculate_proportions(total_data_cluster3, 1, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)
    total_pop4, prevalence_prop4, unaware_prop4, aware_no_art_prop4, aware_art_vls_prop4,_ = calculate_proportions(total_data_cluster4, 1, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)
    total_pop5, prevalence_prop5, unaware_prop5, aware_no_art_prop5, aware_art_vls_prop5,_ = calculate_proportions(total_data_cluster5, 1, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)
    total_pop6, prevalence_prop6, unaware_prop6, aware_no_art_prop6, aware_art_vls_prop6,_ = calculate_proportions(total_data_cluster6, 1, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)
    total_pop7, prevalence_prop7, unaware_prop7, aware_no_art_prop7, aware_art_vls_prop7,_ = calculate_proportions(total_data_cluster7, 1, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)
    total_pop8, prevalence_prop8, unaware_prop8, aware_no_art_prop8, aware_art_vls_prop8,_ = calculate_proportions(total_data_cluster8, 1, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)

    prep_coverage1 = data_array_cluster1[:,2,:,0]*prep_values[cluster1_index,2][:,np.newaxis]
    prep1 = np.round(np.apply_over_axes(np.sum, prep_coverage1, [0,1]).item()/np.apply_over_axes(np.sum, data_array_cluster1[:,2,:,0], [0,1]).item(), 4)
    prep_prop1 = np.array([0,0,prep1])
    
    prep_coverage2 = data_array_cluster2[:,2,:,0]*prep_values[cluster2_index,2][:,np.newaxis]
    prep2 = np.round(np.apply_over_axes(np.sum, prep_coverage2,[0,1]).item()/np.apply_over_axes(np.sum, data_array_cluster2[:,2,:,0], [0,1]).item(), 4)
    prep_prop2 = np.array([0,0,prep2])
    
    prep_coverage3= data_array_cluster3[:,2,:,0]*prep_values[cluster3_index,2][:,np.newaxis]
    prep3= np.round(np.apply_over_axes(np.sum, prep_coverage3, [0,1]).item()/np.apply_over_axes(np.sum, data_array_cluster3[:,2,:,0], [0,1]).item(), 4)
    prep_prop3= np.array([0,0,prep3])
    
    prep_coverage4 = data_array_cluster4[:,2,:,0]*prep_values[cluster4_index,2][:,np.newaxis]
    prep4 = np.round(np.apply_over_axes(np.sum, prep_coverage4, [0,1]).item()/np.apply_over_axes(np.sum, data_array_cluster4[:,2,:,0], [0,1]).item(), 4)
    prep_prop4 = np.array([0,0,prep4])
    
    prep_coverage5 = data_array_cluster5[:,2,:,0]*prep_values[cluster5_index,2][:,np.newaxis]
    prep5 = np.round(np.apply_over_axes(np.sum, prep_coverage5,[0,1]).item()/np.apply_over_axes(np.sum, data_array_cluster5[:,2,:,0], [0,1]).item(), 4)
    prep_prop5 = np.array([0,0,prep5])
    
    prep_coverage6 = data_array_cluster6[:,2,:,0]*prep_values[cluster6_index,2][:,np.newaxis]
    prep6= np.round(np.apply_over_axes(np.sum, prep_coverage6, [0,1]).item()/np.apply_over_axes(np.sum, data_array_cluster6[:,2,:,0], [0,1]).item(), 4)
    prep_prop6= np.array([0,0,prep6])
    
    prep_coverage7 = data_array_cluster7[:,2,:,0]*prep_values[cluster7_index,2][:,np.newaxis]
    prep7 = np.round(np.apply_over_axes(np.sum, prep_coverage7, [0,1]).item()/np.apply_over_axes(np.sum, data_array_cluster7[:,2,:,0], [0,1]).item(), 4)
    prep_prop7 = np.array([0,0,prep7])
    
    prep_coverage8 = data_array_cluster8[:,2,:,0]*prep_values[cluster8_index,2][:,np.newaxis]
    prep8 = np.round(np.apply_over_axes(np.sum, prep_coverage8,[0,1]).item()/np.apply_over_axes(np.sum, data_array_cluster8[:,2,:,0], [0,1]).item(), 4)
    prep_prop8 = np.array([0,0,prep8])
    
    
    current_state1 = np.transpose(np.vstack((prevalence_prop1, unaware_prop1, aware_no_art_prop1, aware_art_vls_prop1, prep_prop1)))
    current_state2 = np.transpose(np.vstack((prevalence_prop2, unaware_prop2, aware_no_art_prop2, aware_art_vls_prop2, prep_prop2)))
    current_state3 = np.transpose(np.vstack((prevalence_prop3, unaware_prop3, aware_no_art_prop3, aware_art_vls_prop3, prep_prop3)))
    current_state4 = np.transpose(np.vstack((prevalence_prop4, unaware_prop4, aware_no_art_prop4, aware_art_vls_prop4, prep_prop4)))
    current_state5 = np.transpose(np.vstack((prevalence_prop5, unaware_prop5, aware_no_art_prop5, aware_art_vls_prop5, prep_prop5)))
    current_state6 = np.transpose(np.vstack((prevalence_prop6, unaware_prop6, aware_no_art_prop6, aware_art_vls_prop6, prep_prop6)))
    current_state7 = np.transpose(np.vstack((prevalence_prop7, unaware_prop7, aware_no_art_prop7, aware_art_vls_prop7, prep_prop7)))
    current_state8 = np.transpose(np.vstack((prevalence_prop8, unaware_prop8, aware_no_art_prop8, aware_art_vls_prop8, prep_prop8)))
    
    return current_state1, current_state2, current_state3, current_state4, current_state5, current_state6, current_state7, current_state8


In [ ]:
# initial_data = data_array_cluster.copy()

def initial_state(initial_data, prep_values):
    state1,state2,state3,state4,state5,state6,state7,state8 = extract_state(initial_data, prep_values)
    time = 0
    return initial_data,state1,state2,state3,state4,state5,state6,state7,state8, prep_values, time